In [1]:
%%capture
#@title Instalamos gym
!pip install swig
!pip install "gymnasium[box2d]"
!pip install numpy
!pip install matplotlib
!pip install tqdm
#!pip install "gymnasium[toy-text]"
!pip install Box2D
!pip install pygame

In [15]:
#@title Importamos librerias
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gymnasium as gym
import Box2D
from typing import Dict, Any
import random


In [16]:
#@title Importamos el lago helado
name = 'Taxi-v3'

# Creamos el entorno Taxi-v3
env = gym.make(name, render_mode="ansi")


In [17]:
class AgentMonteCarloOnPolicy:
    def __init__(self, env: gym.Env, hyperparameters: Dict = None):
        """
        Inicializa el agente Monte Carlo On-Policy
        
        Args:
            env: Entorno de Gymnasium
            hyperparameters: Diccionario con los hiperparámetros (opcional)
        """
        self.env = env
        
        # Hiperparámetros con valores por defecto
        if hyperparameters is None:
            hyperparameters = {}
        
        self.gamma = hyperparameters.get('gamma', 0.99)
        self.epsilon = hyperparameters.get('epsilon', 1.0)
        self.epsilon_min = hyperparameters.get('epsilon_min', 0.01)
        self.epsilon_decay = hyperparameters.get('epsilon_decay', 0.995)
        
        # Tablas para el aprendizaje
        self.q_table = {}  # Valores Q: Q(s,a)
        self.returns = {}  # Almacena retornos para cada par estado-acción
        
        # Buffer para almacenar el episodio actual
        self.episode_buffer = []
        
        # Métricas de aprendizaje
        self.episode_rewards = []
        self.episode_lengths = []
        self.current_episode_reward = 0
        self.current_episode_length = 0
        
    def get_action(self, state: Any) -> Any:
        """
        Política epsilon-soft para selección de acciones
        
        Args:
            state: Estado actual del entorno
        
        Returns:
            action: Acción seleccionada
        """
        state_key = tuple(state) if isinstance(state, np.ndarray) else state
        
        # Inicializar estado si no existe
        if state_key not in self.q_table:
            self.q_table[state_key] = {
                action: 0.0 for action in range(self.env.action_space.n)
            }
        
        # Política epsilon-soft
        if random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            return max(self.q_table[state_key].items(), key=lambda x: x[1])[0]
    
    def update(self, state: Any, action: Any, next_state: Any, 
               reward: float, terminated: bool, truncated: bool, info: Dict):
        """
        Actualiza el agente con la nueva transición
        
        Args:
            state: Estado actual
            action: Acción tomada
            next_state: Siguiente estado
            reward: Recompensa obtenida
            terminated: Si el episodio terminó naturalmente
            truncated: Si el episodio fue truncado
            info: Información adicional del entorno
        """
        # Almacenar la transición en el buffer del episodio
        self.episode_buffer.append((state, action, reward))
        
        # Actualizar métricas del episodio actual
        self.current_episode_reward += reward
        self.current_episode_length += 1
        
        # Si el episodio terminó, procesar el episodio completo
        if terminated or truncated:
            self._process_episode()
            
            # Guardar métricas del episodio
            self.episode_rewards.append(self.current_episode_reward)
            self.episode_lengths.append(self.current_episode_length)
            
            # Resetear métricas y buffer para el siguiente episodio
            self.current_episode_reward = 0
            self.current_episode_length = 0
            self.episode_buffer = []
            
            # Actualizar epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def _process_episode(self):
        """
        Procesa el episodio completo usando Monte Carlo first-visit
        """
        # Calcular retornos para cada paso
        G = 0
        returns = []
        
        # Calcular retornos desde el final del episodio
        for _, _, reward in reversed(self.episode_buffer):
            G = reward + self.gamma * G
            returns.insert(0, G)
        
        # Actualizar valores Q para cada paso (first-visit)
        states_actions_seen = set()
        
        for idx, ((state, action, _), G) in enumerate(zip(self.episode_buffer, returns)):
            state_key = tuple(state) if isinstance(state, np.ndarray) else state
            sa_pair = (state_key, action)
            
            # Solo actualizar en la primera visita
            if sa_pair not in states_actions_seen:
                states_actions_seen.add(sa_pair)
                
                # Inicializar si es necesario
                if state_key not in self.q_table:
                    self.q_table[state_key] = {
                        a: 0.0 for a in range(self.env.action_space.n)
                    }
                if sa_pair not in self.returns:
                    self.returns[sa_pair] = []
                
                # Actualizar retornos y valor Q
                self.returns[sa_pair].append(G)
                self.q_table[state_key][action] = np.mean(self.returns[sa_pair])
    
    def stats(self) -> Dict:
        """
        Retorna estadísticas del aprendizaje
        
        Returns:
            Dict con estadísticas relevantes
        """
        if not self.episode_rewards:
            return {
                'mean_reward': 0,
                'mean_length': 0,
                'current_epsilon': self.epsilon,
                'n_episodes': 0,
                'q_table_size': len(self.q_table)
            }
            
        return {
            'mean_reward': np.mean(self.episode_rewards[-100:]),  # Media últimos 100 episodios
            'mean_length': np.mean(self.episode_lengths[-100:]),  # Media últimos 100 episodios
            'current_epsilon': self.epsilon,
            'n_episodes': len(self.episode_rewards),
            'q_table_size': len(self.q_table)
        }

In [19]:
# Entrenamiento
n_episodes = 1000
agent = AgentMonteCarloOnPolicy(env)


for episode in tqdm(range(n_episodes)):
    obs, info = env.reset()
    done = False
    
    while not done:
        action = agent.get_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        
        # Actualizar el agente
        agent.update(obs, action, next_obs, reward, terminated, truncated, info)
        
        done = terminated or truncated
        obs = next_obs
    
    # Obtener estadísticas
    if episode % 100 == 0:
        stats = agent.stats()
        print(f"\nEpisodio {episode}:")
        print(f"Reward media (últimos 100): {stats['mean_reward']:.2f}")
        print(f"Longitud media (últimos 100): {stats['mean_length']:.2f}")
        print(f"Epsilon actual: {stats['current_epsilon']:.3f}")

  5%|▌         | 54/1000 [00:00<00:03, 266.86it/s]


Episodio 0:
Reward media (últimos 100): -857.00
Longitud media (últimos 100): 200.00
Epsilon actual: 0.995


 17%|█▋        | 169/1000 [00:00<00:02, 357.86it/s]


Episodio 100:
Reward media (últimos 100): -741.36
Longitud media (últimos 100): 193.35
Epsilon actual: 0.603

Episodio 200:
Reward media (últimos 100): -647.47
Longitud media (últimos 100): 168.61
Epsilon actual: 0.365


 41%|████      | 412/1000 [00:01<00:01, 487.92it/s]

...

 75%|███████▍  | 747/1000 [00:01<00:00, 526.04it/s]


Episodio 700:
Reward media (últimos 100): -261.86
Longitud media (últimos 100): 160.13
Epsilon actual: 0.030


 90%|█████████ | 902/1000 [00:01<00:00, 466.81it/s]


Episodio 800:
Reward media (últimos 100): -211.94
Longitud media (últimos 100): 150.56
Epsilon actual: 0.018

Episodio 900:
Reward media (últimos 100): -177.62
Longitud media (últimos 100): 169.16
Epsilon actual: 0.011


100%|██████████| 1000/1000 [00:02<00:00, 451.86it/s]


In [20]:
obs

423